In [9]:
import random
import matplotlib.pyplot as plt
from scipy.stats import norm, stats
import numpy as np
from statistics import mean

In [10]:
# Same as original
def generate_obs(x):
    "Return a single point from the distributino"
    return (0.3 * norm(35, 5).pdf(x)  + \
            0.7 * norm(60, 5).pdf(x)) * \
                200

# Same as original
def query_data(lower=20, upper=80, n_obs=100, verbose=False, seed=1):
    " Generate a list of results of the generate_obs() function from random inputs"
    
    random.seed(seed)
    
    # Use random.uniform() to create random values and create a list
    observations = []
    x = [random.uniform(lower, upper) for _ in range(n_obs)]
    
    # Sort the list.  That will make our plot actually readable.
    #   NB:  x.sort() returns None so we can't add sort() to our definition of x
    x.sort()
    
    # Runs through our random inputs and remenbers the results
    for xval in x:                
        obs = generate_obs(xval)
        observations.append(obs) 
        if verbose:  
            print(f"f({xval:2.3f}) = {obs:2.3f}")
            
    return x, observations  

# Same as original
def get_max_xy(xy_arr):
    " Input is two-col ndarray.  Cols: [x, f(x)].  Returns x and f(x) at max f(x) value."
    
    max_row = xy_arr[np.where(xy[:,1] == np.max(xy_arr[:,1]))][0]
    x_max, y_max = max_row[0], max_row[1]

    return x_max, y_max

# Same as original
def next_x_vals(guess, frame=3, step=1, direction='up', samples=None):
    """Create and return a vector such that each element is 'step' different
         from the last, starting at guess +/- step, depending on the 
         'direction'.  The length of the vector is 'frame'.  """
    
    return_list = []
    for ix in range(frame ):
        if direction == 'up':
            return_list.append(guess + ix * step)
        else:
            return_list.append(guess - ix * step)
            
    return return_list
            
 # Same as original   
def next_y_vals(next_x_vals, func):
    """ Given a vector of next_x_vals and a function, create and return a vector of f(x)."""
    
    return_vector = []
    
    for x in next_x_vals:
        return_vector.append(func(x))        
    return return_vector

def ORIGINAL_gradient_search_max(guess, func, lower_bound, upper_bound, step, frame, direction=None):
    """Conduct a gradient search to find max value of a solution space."""  
    
    DEBUG = False
    
    # Remember the oringinal guess
    original_guess = guess
    original_result = func(original_guess)
    if DEBUG: print(f"current: x {original_guess:.2f}   y {original_result:.2f}")
    
    # Find x values for a few steps higher and lower than current guess.
    x_going_up = next_x_vals(guess, frame, step, 'up')
    x_going_down = next_x_vals(guess, frame, step, 'down')
    if DEBUG: print(x_going_up, x_going_down)
    
    
    # Find corresponding mean y values
    y_going_up = mean(next_y_vals(x_going_up, func))
    y_going_down = mean(next_y_vals(x_going_down, func))
    if DEBUG: print(f"y_going_down: {y_going_down:.2f}  y_going_up: {y_going_up:.2f} ")
    
    
    # Figure out if we're going up or down
    if direction is None:
        if y_going_up >= original_result and y_going_up >= y_going_down:
            direction = 'up'
        if y_going_down > original_result and y_going_up < y_going_down:
            direction = 'down' 
        print(f"We're searching {direction}.")
    
    # Adjust the guess as needed
    if direction == 'up' and y_going_up >= original_result:
        guess = original_guess + step
    elif direction == 'down'  and y_going_down >= original_result:
        guess = original_guess - step
            
    # We'll keep going if the guess changed and it's within bounds. Otherwise we're done.
    if guess != original_guess and guess >= lower_bound and guess <= upper_bound:
        print(f"Current answer:  x: {guess:.2f}  y: {func(guess):.2f}")
        gradient_search_max(guess, func, lower_bound, upper_bound, step, frame, direction=direction)
        
    else:
        print(f"Final answer:  x: {guess:.2f}  y: {func(guess):.2f}")
        return guess

In [11]:
# Create our actual PDF (probability density function)
X_plot = np.linspace(10, 100, 1000)[:, np.newaxis]

true_dens = (0.3 * norm(35, 5).pdf(X_plot[:, 0])  + \
             0.7 * norm(60, 5).pdf(X_plot[:, 0])) * \
            200

In [12]:
# Set our search range
n_obs = 10    
lower = 20
upper = 75

# Query the data
x, obs = query_data (lower=lower, upper=upper, n_obs=n_obs, verbose=False) 
obs_array = np.array(obs)

# Get an array of the x,y values
xy = np.stack((np.array(x), obs_array), axis=1)

# Figure out the values of x and y where y is at its max
x_max, y_max = get_max_xy(xy)

# Set some initial values
guess0 = x_max
result0 = y_max
step = 1
frame = 3
guess = x_max

In [13]:
# Upgraded
def gradient_search_max(guess, func, lower_bound, upper_bound, step, frame, direction=None, 
                        x_val_picker=next_x_vals, samples=10, DEBUG=False):
    """Conduct a gradient search to find max value of a solution space.
    
      This is changed to accept a keyword argument for the function x_val_picker().  This
        allows us to swith the algoritym used to come up with our sampling for the hill-
         climbing.  

      We've also added a keyword argument for samples.  We'll want that when
        we do random sampling of the neighborhood around our current guess.

      Because we have potentially a lot of output, we moved DEBUG to a keyword argument
        so it will be easier to switch on or off.  
    """  
    
    # Remember the oringinal guess
    original_guess = guess
    original_result = func(original_guess)
    if DEBUG: print(f"\n\nstart: x {original_guess:.2f}   y {original_result:.2f}")
    
    # Find x values for a few steps higher and lower than current guess.   
    
    x_going_up = x_val_picker(guess, frame=frame, samples=samples, direction='up', step=step)
    x_going_down = x_val_picker(guess, frame, samples=samples, direction='down', step=step)
    if DEBUG: print(x_going_up, x_going_down)
      
    # Find corresponding mean y values
    y_going_up = mean(next_y_vals(x_going_up, func))
    y_going_down = mean(next_y_vals(x_going_down, func))
    if DEBUG: print(f"y_going_down: {y_going_down:.2f}  y_going_up: {y_going_up:.2f} ")
    
    
    # Figure out if we're going up or down
    if direction is None:
        if y_going_up >= original_result and y_going_up >= y_going_down:
            direction = 'up'
        if y_going_down > original_result and y_going_up < y_going_down:
            direction = 'down' 
        print(f"We're searching {direction}.")
    
    # Adjust the guess as needed
    if direction == 'up' and y_going_up >= original_result:
        guess = original_guess + step
    elif direction == 'down'  and y_going_down >= original_result:
        guess = original_guess - step
            
    # We'll keep going if the guess changed and it's within bounds. Otherwise we're done.
    if guess != original_guess and guess >= lower_bound and guess <= upper_bound:
        print(f"Current answer:  x: {guess:.2f}  y: {func(guess):.2f}")
        gradient_search_max(guess, func, lower_bound, upper_bound, step, frame, direction=direction, 
                        x_val_picker=x_val_picker, samples=samples, DEBUG=DEBUG)
        
    else:
        print(f"Final answer:  x: {guess:.2f}  y: {func(guess):.2f}")
        return guess

    
func = generate_obs
lower_bound_gradient = 0
upper_bound_gradient = 100
step = .1
frame = 3


# This runs the gradient search in the original mode
gradient_search_max(guess, func, lower_bound_gradient, upper_bound_gradient, 
                    step, frame,x_val_picker=next_x_vals, DEBUG=False)

We're searching down.
Current answer:  x: 61.91  y: 10.39
Current answer:  x: 61.81  y: 10.46
Current answer:  x: 61.71  y: 10.54
Current answer:  x: 61.61  y: 10.61
Current answer:  x: 61.51  y: 10.67
Current answer:  x: 61.41  y: 10.74
Current answer:  x: 61.31  y: 10.79
Current answer:  x: 61.21  y: 10.85
Current answer:  x: 61.11  y: 10.90
Current answer:  x: 61.01  y: 10.95
Current answer:  x: 60.91  y: 10.99
Current answer:  x: 60.81  y: 11.03
Current answer:  x: 60.71  y: 11.06
Current answer:  x: 60.61  y: 11.09
Current answer:  x: 60.51  y: 11.11
Current answer:  x: 60.41  y: 11.13
Current answer:  x: 60.31  y: 11.15
Current answer:  x: 60.21  y: 11.16
Current answer:  x: 60.11  y: 11.17
Current answer:  x: 60.01  y: 11.17
Final answer:  x: 60.01  y: 11.17


In [18]:
# New Hill-climbing Data

# Here, we introduce the alternative data.   We take samples of the independent variable 'x'
# over the range of the 'frame'.

In [19]:
def next_x_vals_rand(guess, frame=3, samples=100, direction='up', step=None):    
    """Create and return a vector of <sample> random values in a range
         <guess> +/- <frame>, depending on the direction.  
    """
    if direction == 'up':
        return [ random.uniform(guess, guess+frame) for _ in range(samples)  ]   
    else:
        return [ random.uniform(guess, guess-frame) for _ in range(samples)  ]

Now we can call the upgraded gradient_search_max() using the new sampling frame:

In [20]:
gradient_search_max(guess, func, lower_bound_gradient, upper_bound_gradient, 
                    step, frame, x_val_picker=next_x_vals_rand, DEBUG=False)

We're searching down.
Current answer:  x: 61.91  y: 10.39
Current answer:  x: 61.81  y: 10.46
Current answer:  x: 61.71  y: 10.54
Current answer:  x: 61.61  y: 10.61
Current answer:  x: 61.51  y: 10.67
Current answer:  x: 61.41  y: 10.74
Current answer:  x: 61.31  y: 10.79
Current answer:  x: 61.21  y: 10.85
Current answer:  x: 61.11  y: 10.90
Final answer:  x: 61.11  y: 10.90


In [16]:
# The actual max of the true PDF:
print(f"true max value: {stats.describe(true_dens).minmax[1]:.2f}")

true max value: 11.17
